# Evolution of a fan

This notebook reproduces the [fan example](https://fastscape-lem.github.io/fastscapelib-fortran/#_fan_f90) provided in the fastscapelib-fortran library. It illustrates continental transport/deposition.

In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import fastscape

%matplotlib inline

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

## Import, customize and inspect the model

A sediment model is available in [fastscape](https://fastscape.readthedocs.io/en/latest/).

In [ ]:
from fastscape.models import sediment_model

We want to model the evolution of a passive escarpment, thus with no uplift.

In [ ]:
from fastscape.processes import Escarpment


model = (sediment_model
         .drop_processes('uplift')
         .update_processes({'init_topography': Escarpment}))

In [ ]:
model

In [ ]:
sediment_model.visualize(show_inputs=True)

## Model setup

In [ ]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.arange(0, 4e5 + 2e3, 2e3),
        'out': np.arange(0, 4e5 + 4e3, 4e3),
    },
    master_clock='time',
    input_vars={
        'grid__shape': ('shape_yx', [101, 201]),
        'grid__length': ('shape_yx', [1e4, 2e4]),
        'boundary__status': ('border', ['fixed_value', 'whatever', 'looped', 'looped']),
        'init_topography': {
            'x_left': 1e4,
            'x_right': 1e4,
            'elevation_left': 0.,
            'elevation_right': 1e3
        },
        'flow__slope_exp': 1.,
        'spl': {
            'k_coef_bedrock': 1e-4,
            'k_coef_soil': 1.5e-4,
            'g_coef_bedrock': 1.,
            'g_coef_soil': 1.,
            'area_exp': 0.4,
            'slope_exp': 1.
        },
        'diffusion': {
            'diffusivity_bedrock': 1e-2,
            'diffusivity_soil': 1.5e-2
        }
    },
    output_vars={
        'out': ['topography__elevation',
                'drainage__area',
                'erosion__rate'],
        None: ['boundary__border',
               'grid__x',
               'grid__y',
               'grid__spacing'],
    }
)

in_ds

## Run the model

This should take about 10 seconds...

In [ ]:
out_ds = (in_ds.xsimlab.run(model=model)
               .set_index(x='grid__x', y='grid__y',
                          border='boundary__border'))

In [ ]:
out_ds

## Plot the outputs

The following plots (total erosion/deposition rate, drainage area and topography cross-section) highlight the dynamics on the left area (depositional) and its interaction with the right area (erosional).

In [ ]:
import hvplot.xarray
import holoviews as hv
from xshade import hillshade


erosion_plot = out_ds.erosion__rate.hvplot.image(
    x='x', y='y', clim=(-2.5e-3, 2.5e-3),
    cmap=plt.cm.RdYlGn_r, groupby='out'
)

hillshade_plot = hillshade(out_ds, 'out').hvplot.image(
    x='x', y='y', cmap=plt.cm.gray, alpha=0.4,
    colorbar=False, hover=False, groupby='out'
)

ysections =  out_ds.topography__elevation.sel(y=[2.5e3, 5e3, 7.5e3])

sections_plot = ysections.hvplot.line(
    by='y', groupby='out', ylim=(0, 1100),
    height=200, legend='top_left',
)

hv.Layout((erosion_plot * hillshade_plot) + sections_plot).cols(1)